In [1]:
import mxnet as mx
from mxnet.gluon import nn
from mxnet.gluon.contrib import nn as nn_contrib
from mxnet import nd
from mxnet import gluon
import numpy as np
ctx = mx.gpu()

In [2]:
global alpha
alpha = 0.5
num_filter = int(32*alpha)

In [3]:
def dp_layer(nfilters, stride, expension_constant):
    out = nn.HybridSequential()
    out.add(nn.Conv2D(nfilters, 3, strides=stride, padding=1, groups=nfilters, use_bias=False))
    out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
    out.add(nn.Activation('relu'))
    out.add(nn.Conv2D(nfilters*expension_constant, 1, strides=1, padding=0, use_bias=False))
    out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
    out.add(nn.Activation('relu'))
    return out

In [4]:
def s16():
    out = nn.HybridSequential()
    with out.name_scope():
        # conv2d
        out.add(nn.Conv2D(num_filter, kernel_size=3, strides=2, padding=1, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        
        # (3) LinearBottleneck
        out.add(dp_layer(num_filter, 1, 1))
        #out.add(nn.Conv2D(num_filter, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter, kernel_size=3, strides=1, padding=1, groups=num_filter, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter/2, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        
        # (4) LinearBottleneck
        out.add(dp_layer(num_filter/2, 1, 6))
        #out.add(nn.Conv2D(num_filter*3, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu')) # conv2_2_linear_scale
        out.add(nn.Conv2D(num_filter*3, kernel_size=3, strides=2, padding=1, groups=num_filter*3, use_bias=False))

        #out.load_parameters("process/mobilenet_v2_0_5_s16.params")
        return out

In [5]:
def s32():
    out = nn.HybridSequential()
    with out.name_scope():
        # (4) LinearBottleneck con't
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*3/4, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1)) # conv2_2_linear_scale
        
        # (5) LinearBottleneck
        out.add(dp_layer(num_filter*3/4, 1, 6))
        #out.add(nn.Conv2D(num_filter*9/2, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*9/2, kernel_size=3, strides=1, padding=1, groups=num_filter*9/2, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*6/8, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1)) # conv2_2_linear_scale concatenate

        # (6) LinearBottleneck
        out.add(dp_layer(num_filter*6/8, 1, 6))
        #out.add(nn.Conv2D(num_filter*9/2, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*9/2, kernel_size=3, strides=2, padding=1, groups=num_filter*9/2, use_bias=False))
        
        #out.load_parameters("process/mobilenet_v2_0_5_s32.params")
        return out

In [6]:
def fc():
    out = nn.HybridSequential()
    with out.name_scope():
        # (6) LinearBottleneck con't
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        
        # (7) LinearBottleneck
        out.add(dp_layer(num_filter, 1, 6))
        #out.add(nn.Conv2D(num_filter*6, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu')) # conv3_2_linear_scale
        out.add(nn.Conv2D(num_filter*6, kernel_size=3, strides=1, padding=1, groups=num_filter*6, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        
        # (8) LinearBottleneck
        out.add(dp_layer(num_filter, 1, 6))
        #out.add(nn.Conv2D(num_filter*6, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*6, kernel_size=3, strides=1, padding=1, groups=num_filter*6, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        
        # (9) LinearBottleneck
        out.add(dp_layer(num_filter, 1, 6))
        #out.add(nn.Conv2D(num_filter*6, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu')) # block_4_1
        out.add(nn.Conv2D(num_filter*6, kernel_size=3, strides=1,padding=1, groups=num_filter*6, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*2, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1)) # block_4_1 concanetate
        
        # (10) LinearBottleneck
        out.add(dp_layer(num_filter*2, 1, 6))
        #out.add(nn.Conv2D(num_filter*12, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu')) 
        out.add(nn.Conv2D(num_filter*12, kernel_size=3, strides=1, padding=1, groups=num_filter*12, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*2, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        
        # (11) LinearBottleneck
        out.add(dp_layer(num_filter*2, 1, 6))
        #out.add(nn.Conv2D(num_filter*12, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*12, kernel_size=3, strides=1, padding=1, groups=num_filter*12, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*2, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        
        # (12) LinearBottleneck
        out.add(dp_layer(num_filter*2, 1, 6))
        #out.add(nn.Conv2D(num_filter*12, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*12, kernel_size=3, strides=1, padding=1, groups=num_filter*12, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*2, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))

        # (13) LinearBottleneck
        out.add(dp_layer(num_filter*2, 1, 6))
        #out.add(nn.Conv2D(num_filter*12, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*12, kernel_size=3, strides=2,padding=1, groups=num_filter*12, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*3, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        
        # (14) LinearBottleneck
        out.add(dp_layer(num_filter*3, 1, 6))
        #out.add(nn.Conv2D(num_filter*18, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*18, kernel_size=3, strides=1, padding=1, groups=num_filter*18, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*3, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        
        # (15) LinearBottleneck
        out.add(dp_layer(num_filter*3, 1, 6))
        #out.add(nn.Conv2D(num_filter*18, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*18, kernel_size=3, strides=1, padding=1, groups=num_filter*18, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*3, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        
        # (16) LinearBottleneck
        out.add(dp_layer(num_filter*3, 1, 6))
        #out.add(nn.Conv2D(num_filter*18, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*18, kernel_size=3, strides=2, padding=1, groups=num_filter*18, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*5, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        
        # (17) LinearBottleneck
        out.add(dp_layer(num_filter*5, 1, 6))
        #out.add(nn.Conv2D(num_filter*30, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*30, kernel_size=3, strides=1, padding=1, groups=num_filter*30, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*5, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        
        # (18) LinearBottleneck
        out.add(dp_layer(num_filter*5, 1, 6))
        #out.add(nn.Conv2D(num_filter*30, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*30, kernel_size=3, strides=1, padding=1, groups=num_filter*30, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*5, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        
        # (19) LinearBottleneck
        out.add(dp_layer(num_filter*5, 1, 6))
        #out.add(nn.Conv2D(num_filter*30, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*30, kernel_size=3, strides=1, padding=1, groups=num_filter*30, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.Conv2D(num_filter*10, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        
        out.add(nn.Conv2D(num_filter*80, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
        out.add(nn.Activation('relu'))
        out.add(nn.GlobalAvgPool2D())
        
        out.add(nn.Conv2D(1000, kernel_size=1, strides=1, padding=0, use_bias=False))
        out.add(nn.Flatten())
        #out.load_parameters("process/mobilenet_v2_0_5_fc.params")
        return out

In [7]:
s16 = s16()
s32 = s32()
fc = fc()
s16.initialize(mx.init.Xavier(magnitude=2))
s32.initialize(mx.init.Xavier(magnitude=2))
fc.initialize(mx.init.Xavier(magnitude=2))
x = nd.zeros((2, 3, 512, 512))
fc(s32(s16(x)))


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<NDArray 2x1000 @cpu(0)>

In [8]:
s16.save_parameters("process/s16.params")
s32.save_parameters("process/s32.params")
fc.save_parameters("process/fc.params")

In [9]:
s16_org_params = mx.nd.load("process/s16.params")
sorted(s16_org_params.keys())

['0.weight',
 '1.beta',
 '1.gamma',
 '1.running_mean',
 '1.running_var',
 '10.beta',
 '10.gamma',
 '10.running_mean',
 '10.running_var',
 '11.0.weight',
 '11.1.beta',
 '11.1.gamma',
 '11.1.running_mean',
 '11.1.running_var',
 '11.3.weight',
 '11.4.beta',
 '11.4.gamma',
 '11.4.running_mean',
 '11.4.running_var',
 '12.beta',
 '12.gamma',
 '12.running_mean',
 '12.running_var',
 '14.weight',
 '3.0.weight',
 '3.1.beta',
 '3.1.gamma',
 '3.1.running_mean',
 '3.1.running_var',
 '3.3.weight',
 '3.4.beta',
 '3.4.gamma',
 '3.4.running_mean',
 '3.4.running_var',
 '4.beta',
 '4.gamma',
 '4.running_mean',
 '4.running_var',
 '6.weight',
 '7.beta',
 '7.gamma',
 '7.running_mean',
 '7.running_var',
 '9.weight']

In [10]:
s32_org_params = mx.nd.load("process/s32.params")
sorted(s32_org_params.keys())

['0.beta',
 '0.gamma',
 '0.running_mean',
 '0.running_var',
 '10.weight',
 '11.beta',
 '11.gamma',
 '11.running_mean',
 '11.running_var',
 '12.0.weight',
 '12.1.beta',
 '12.1.gamma',
 '12.1.running_mean',
 '12.1.running_var',
 '12.3.weight',
 '12.4.beta',
 '12.4.gamma',
 '12.4.running_mean',
 '12.4.running_var',
 '13.beta',
 '13.gamma',
 '13.running_mean',
 '13.running_var',
 '15.weight',
 '2.weight',
 '3.beta',
 '3.gamma',
 '3.running_mean',
 '3.running_var',
 '4.0.weight',
 '4.1.beta',
 '4.1.gamma',
 '4.1.running_mean',
 '4.1.running_var',
 '4.3.weight',
 '4.4.beta',
 '4.4.gamma',
 '4.4.running_mean',
 '4.4.running_var',
 '5.beta',
 '5.gamma',
 '5.running_mean',
 '5.running_var',
 '7.weight',
 '8.beta',
 '8.gamma',
 '8.running_mean',
 '8.running_var']

In [11]:
fc_org_params = mx.nd.load("process/fc.params")
sorted(fc_org_params.keys())

['0.beta',
 '0.gamma',
 '0.running_mean',
 '0.running_var',
 '10.weight',
 '100.0.weight',
 '100.1.beta',
 '100.1.gamma',
 '100.1.running_mean',
 '100.1.running_var',
 '100.3.weight',
 '100.4.beta',
 '100.4.gamma',
 '100.4.running_mean',
 '100.4.running_var',
 '101.beta',
 '101.gamma',
 '101.running_mean',
 '101.running_var',
 '103.weight',
 '104.beta',
 '104.gamma',
 '104.running_mean',
 '104.running_var',
 '106.weight',
 '107.beta',
 '107.gamma',
 '107.running_mean',
 '107.running_var',
 '108.weight',
 '109.beta',
 '109.gamma',
 '109.running_mean',
 '109.running_var',
 '11.beta',
 '11.gamma',
 '11.running_mean',
 '11.running_var',
 '112.weight',
 '12.0.weight',
 '12.1.beta',
 '12.1.gamma',
 '12.1.running_mean',
 '12.1.running_var',
 '12.3.weight',
 '12.4.beta',
 '12.4.gamma',
 '12.4.running_mean',
 '12.4.running_var',
 '13.beta',
 '13.gamma',
 '13.running_mean',
 '13.running_var',
 '15.weight',
 '16.beta',
 '16.gamma',
 '16.running_mean',
 '16.running_var',
 '18.weight',
 '19.beta',


In [12]:
mn_org_params = mx.nd.load("process/mobilenetv2_0.5-aabd26cd.params")
sorted(mn_org_params.keys())

['features.0.weight',
 'features.1.beta',
 'features.1.gamma',
 'features.1.running_mean',
 'features.1.running_var',
 'features.10.out.0.weight',
 'features.10.out.1.beta',
 'features.10.out.1.gamma',
 'features.10.out.1.running_mean',
 'features.10.out.1.running_var',
 'features.10.out.3.weight',
 'features.10.out.4.beta',
 'features.10.out.4.gamma',
 'features.10.out.4.running_mean',
 'features.10.out.4.running_var',
 'features.10.out.6.weight',
 'features.10.out.7.beta',
 'features.10.out.7.gamma',
 'features.10.out.7.running_mean',
 'features.10.out.7.running_var',
 'features.11.out.0.weight',
 'features.11.out.1.beta',
 'features.11.out.1.gamma',
 'features.11.out.1.running_mean',
 'features.11.out.1.running_var',
 'features.11.out.3.weight',
 'features.11.out.4.beta',
 'features.11.out.4.gamma',
 'features.11.out.4.running_mean',
 'features.11.out.4.running_var',
 'features.11.out.6.weight',
 'features.11.out.7.beta',
 'features.11.out.7.gamma',
 'features.11.out.7.running_mean'

In [13]:
s16_org_params['0.weight']=mn_org_params['features.0.weight']
s16_org_params['1.beta']=mn_org_params['features.1.beta']
s16_org_params['1.gamma']=mn_org_params['features.1.gamma']
s16_org_params['1.running_mean']=mn_org_params['features.1.running_mean']
s16_org_params['1.running_var']=mn_org_params['features.1.running_var']
s16_org_params['4.beta']=mn_org_params['features.3.out.1.beta']
s16_org_params['4.gamma']=mn_org_params['features.3.out.1.gamma']
s16_org_params['4.running_mean']=mn_org_params['features.3.out.1.running_mean']
s16_org_params['4.running_var']=mn_org_params['features.3.out.1.running_var']
s16_org_params['6.weight']=mn_org_params['features.3.out.3.weight']
s16_org_params['7.beta']=mn_org_params['features.3.out.4.beta']
s16_org_params['7.gamma']=mn_org_params['features.3.out.4.gamma']
s16_org_params['7.running_mean']=mn_org_params['features.3.out.4.running_mean']
s16_org_params['7.running_var']=mn_org_params['features.3.out.4.running_var']
s16_org_params['9.weight']=mn_org_params['features.3.out.6.weight']
s16_org_params['10.beta']=mn_org_params['features.3.out.7.beta']
s16_org_params['10.gamma']=mn_org_params['features.3.out.7.gamma']
s16_org_params['10.running_mean']=mn_org_params['features.3.out.7.running_mean']
s16_org_params['10.running_var']=mn_org_params['features.3.out.7.running_var']

s16_org_params['12.beta']=mn_org_params['features.4.out.1.beta']
s16_org_params['12.gamma']=mn_org_params['features.4.out.1.gamma']
s16_org_params['12.running_mean']=mn_org_params['features.4.out.1.running_mean']
s16_org_params['12.running_var']=mn_org_params['features.4.out.1.running_var']
s16_org_params['14.weight']=mn_org_params['features.4.out.3.weight']

In [14]:
s32_org_params['0.beta']=mn_org_params['features.4.out.4.beta']
s32_org_params['0.gamma']=mn_org_params['features.4.out.4.gamma']
s32_org_params['0.running_mean']=mn_org_params['features.4.out.4.running_mean']
s32_org_params['0.running_var']=mn_org_params['features.4.out.4.running_var']
s32_org_params['2.weight']=mn_org_params['features.4.out.6.weight']
s32_org_params['3.beta']=mn_org_params['features.4.out.7.beta']
s32_org_params['3.gamma']=mn_org_params['features.4.out.7.gamma']
s32_org_params['3.running_mean']=mn_org_params['features.4.out.7.running_mean']
s32_org_params['3.running_var']=mn_org_params['features.4.out.7.running_var']

s32_org_params['5.beta']=mn_org_params['features.5.out.1.beta']
s32_org_params['5.gamma']=mn_org_params['features.5.out.1.gamma']
s32_org_params['5.running_mean']=mn_org_params['features.5.out.1.running_mean']
s32_org_params['5.running_var']=mn_org_params['features.5.out.1.running_var']
s32_org_params['7.weight']=mn_org_params['features.5.out.3.weight']
s32_org_params['8.beta']=mn_org_params['features.5.out.4.beta']
s32_org_params['8.gamma']=mn_org_params['features.5.out.4.gamma']
s32_org_params['8.running_mean']=mn_org_params['features.5.out.4.running_mean']
s32_org_params['8.running_var']=mn_org_params['features.5.out.4.running_var']
s32_org_params['10.weight']=mn_org_params['features.5.out.6.weight']
s32_org_params['11.beta']=mn_org_params['features.5.out.7.beta']
s32_org_params['11.gamma']=mn_org_params['features.5.out.7.gamma']
s32_org_params['11.running_mean']=mn_org_params['features.5.out.7.running_mean']
s32_org_params['11.running_var']=mn_org_params['features.5.out.7.running_var']

s32_org_params['13.beta']=mn_org_params['features.6.out.1.beta']
s32_org_params['13.gamma']=mn_org_params['features.6.out.1.gamma']
s32_org_params['13.running_mean']=mn_org_params['features.6.out.1.running_mean']
s32_org_params['13.running_var']=mn_org_params['features.6.out.1.running_var']
s32_org_params['15.weight']=mn_org_params['features.6.out.3.weight']

In [15]:
fc_org_params['0.beta']=mn_org_params['features.6.out.4.beta']
fc_org_params['0.gamma']=mn_org_params['features.6.out.4.gamma']
fc_org_params['0.running_mean']=mn_org_params['features.6.out.4.running_mean']
fc_org_params['0.running_var']=mn_org_params['features.6.out.4.running_var']
fc_org_params['2.weight']=mn_org_params['features.6.out.6.weight']
fc_org_params['3.beta']=mn_org_params['features.6.out.7.beta']
fc_org_params['3.gamma']=mn_org_params['features.6.out.7.gamma']
fc_org_params['3.running_mean']=mn_org_params['features.6.out.7.running_mean']
fc_org_params['3.running_var']=mn_org_params['features.6.out.7.running_var']

fc_org_params['5.beta']=mn_org_params['features.7.out.1.beta']
fc_org_params['5.gamma']=mn_org_params['features.7.out.1.gamma']
fc_org_params['5.running_mean']=mn_org_params['features.7.out.1.running_mean']
fc_org_params['5.running_var']=mn_org_params['features.7.out.1.running_var']
fc_org_params['7.weight']=mn_org_params['features.7.out.3.weight']
fc_org_params['8.beta']=mn_org_params['features.7.out.4.beta']
fc_org_params['8.gamma']=mn_org_params['features.7.out.4.gamma']
fc_org_params['8.running_mean']=mn_org_params['features.7.out.4.running_mean']
fc_org_params['8.running_var']=mn_org_params['features.7.out.4.running_var']
fc_org_params['10.weight']=mn_org_params['features.7.out.6.weight']
fc_org_params['11.beta']=mn_org_params['features.7.out.7.beta']
fc_org_params['11.gamma']=mn_org_params['features.7.out.7.gamma']
fc_org_params['11.running_mean']=mn_org_params['features.7.out.7.running_mean']
fc_org_params['11.running_var']=mn_org_params['features.7.out.7.running_var']

fc_org_params['13.beta']=mn_org_params['features.8.out.1.beta']
fc_org_params['13.gamma']=mn_org_params['features.8.out.1.gamma']
fc_org_params['13.running_mean']=mn_org_params['features.8.out.1.running_mean']
fc_org_params['13.running_var']=mn_org_params['features.8.out.1.running_var']
fc_org_params['15.weight']=mn_org_params['features.8.out.3.weight']
fc_org_params['16.beta']=mn_org_params['features.8.out.4.beta']
fc_org_params['16.gamma']=mn_org_params['features.8.out.4.gamma']
fc_org_params['16.running_mean']=mn_org_params['features.8.out.4.running_mean']
fc_org_params['16.running_var']=mn_org_params['features.8.out.4.running_var']
fc_org_params['18.weight']=mn_org_params['features.8.out.6.weight']
fc_org_params['19.beta']=mn_org_params['features.8.out.7.beta']
fc_org_params['19.gamma']=mn_org_params['features.8.out.7.gamma']
fc_org_params['19.running_mean']=mn_org_params['features.8.out.7.running_mean']
fc_org_params['19.running_var']=mn_org_params['features.8.out.7.running_var']

fc_org_params['21.beta']=mn_org_params['features.9.out.1.beta']
fc_org_params['21.gamma']=mn_org_params['features.9.out.1.gamma']
fc_org_params['21.running_mean']=mn_org_params['features.9.out.1.running_mean']
fc_org_params['21.running_var']=mn_org_params['features.9.out.1.running_var']
fc_org_params['23.weight']=mn_org_params['features.9.out.3.weight']
fc_org_params['24.beta']=mn_org_params['features.9.out.4.beta']
fc_org_params['24.gamma']=mn_org_params['features.9.out.4.gamma']
fc_org_params['24.running_mean']=mn_org_params['features.9.out.4.running_mean']
fc_org_params['24.running_var']=mn_org_params['features.9.out.4.running_var']
fc_org_params['26.weight']=mn_org_params['features.9.out.6.weight']
fc_org_params['27.beta']=mn_org_params['features.9.out.7.beta']
fc_org_params['27.gamma']=mn_org_params['features.9.out.7.gamma']
fc_org_params['27.running_mean']=mn_org_params['features.9.out.7.running_mean']
fc_org_params['27.running_var']=mn_org_params['features.9.out.7.running_var']

fc_org_params['29.beta']=mn_org_params['features.10.out.1.beta']
fc_org_params['29.gamma']=mn_org_params['features.10.out.1.gamma']
fc_org_params['29.running_mean']=mn_org_params['features.10.out.1.running_mean']
fc_org_params['29.running_var']=mn_org_params['features.10.out.1.running_var']
fc_org_params['31.weight']=mn_org_params['features.10.out.3.weight']
fc_org_params['32.beta']=mn_org_params['features.10.out.4.beta']
fc_org_params['32.gamma']=mn_org_params['features.10.out.4.gamma']
fc_org_params['32.running_mean']=mn_org_params['features.10.out.4.running_mean']
fc_org_params['32.running_var']=mn_org_params['features.10.out.4.running_var']
fc_org_params['34.weight']=mn_org_params['features.10.out.6.weight']
fc_org_params['35.beta']=mn_org_params['features.10.out.7.beta']
fc_org_params['35.gamma']=mn_org_params['features.10.out.7.gamma']
fc_org_params['35.running_mean']=mn_org_params['features.10.out.7.running_mean']
fc_org_params['35.running_var']=mn_org_params['features.10.out.7.running_var']

fc_org_params['37.beta']=mn_org_params['features.11.out.1.beta']
fc_org_params['37.gamma']=mn_org_params['features.11.out.1.gamma']
fc_org_params['37.running_mean']=mn_org_params['features.11.out.1.running_mean']
fc_org_params['37.running_var']=mn_org_params['features.11.out.1.running_var']
fc_org_params['39.weight']=mn_org_params['features.11.out.3.weight']
fc_org_params['40.beta']=mn_org_params['features.11.out.4.beta']
fc_org_params['40.gamma']=mn_org_params['features.11.out.4.gamma']
fc_org_params['40.running_mean']=mn_org_params['features.11.out.4.running_mean']
fc_org_params['40.running_var']=mn_org_params['features.11.out.4.running_var']
fc_org_params['42.weight']=mn_org_params['features.11.out.6.weight']
fc_org_params['43.beta']=mn_org_params['features.11.out.7.beta']
fc_org_params['43.gamma']=mn_org_params['features.11.out.7.gamma']
fc_org_params['43.running_mean']=mn_org_params['features.11.out.7.running_mean']
fc_org_params['43.running_var']=mn_org_params['features.11.out.7.running_var']

fc_org_params['45.beta']=mn_org_params['features.12.out.1.beta']
fc_org_params['45.gamma']=mn_org_params['features.12.out.1.gamma']
fc_org_params['45.running_mean']=mn_org_params['features.12.out.1.running_mean']
fc_org_params['45.running_var']=mn_org_params['features.12.out.1.running_var']
fc_org_params['47.weight']=mn_org_params['features.12.out.3.weight']
fc_org_params['48.beta']=mn_org_params['features.12.out.4.beta']
fc_org_params['48.gamma']=mn_org_params['features.12.out.4.gamma']
fc_org_params['48.running_mean']=mn_org_params['features.12.out.4.running_mean']
fc_org_params['48.running_var']=mn_org_params['features.12.out.4.running_var']
fc_org_params['50.weight']=mn_org_params['features.12.out.6.weight']
fc_org_params['51.beta']=mn_org_params['features.12.out.7.beta']
fc_org_params['51.gamma']=mn_org_params['features.12.out.7.gamma']
fc_org_params['51.running_mean']=mn_org_params['features.12.out.7.running_mean']
fc_org_params['51.running_var']=mn_org_params['features.12.out.7.running_var']

fc_org_params['53.beta']=mn_org_params['features.13.out.1.beta']
fc_org_params['53.gamma']=mn_org_params['features.13.out.1.gamma']
fc_org_params['53.running_mean']=mn_org_params['features.13.out.1.running_mean']
fc_org_params['53.running_var']=mn_org_params['features.13.out.1.running_var']
fc_org_params['55.weight']=mn_org_params['features.13.out.3.weight']
fc_org_params['56.beta']=mn_org_params['features.13.out.4.beta']
fc_org_params['56.gamma']=mn_org_params['features.13.out.4.gamma']
fc_org_params['56.running_mean']=mn_org_params['features.13.out.4.running_mean']
fc_org_params['56.running_var']=mn_org_params['features.13.out.4.running_var']
fc_org_params['58.weight']=mn_org_params['features.13.out.6.weight']
fc_org_params['59.beta']=mn_org_params['features.13.out.7.beta']
fc_org_params['59.gamma']=mn_org_params['features.13.out.7.gamma']
fc_org_params['59.running_mean']=mn_org_params['features.13.out.7.running_mean']
fc_org_params['59.running_var']=mn_org_params['features.13.out.7.running_var']

fc_org_params['61.beta']=mn_org_params['features.14.out.1.beta']
fc_org_params['61.gamma']=mn_org_params['features.14.out.1.gamma']
fc_org_params['61.running_mean']=mn_org_params['features.14.out.1.running_mean']
fc_org_params['61.running_var']=mn_org_params['features.14.out.1.running_var']
fc_org_params['63.weight']=mn_org_params['features.14.out.3.weight']
fc_org_params['64.beta']=mn_org_params['features.14.out.4.beta']
fc_org_params['64.gamma']=mn_org_params['features.14.out.4.gamma']
fc_org_params['64.running_mean']=mn_org_params['features.14.out.4.running_mean']
fc_org_params['64.running_var']=mn_org_params['features.14.out.4.running_var']
fc_org_params['66.weight']=mn_org_params['features.14.out.6.weight']
fc_org_params['67.beta']=mn_org_params['features.14.out.7.beta']
fc_org_params['67.gamma']=mn_org_params['features.14.out.7.gamma']
fc_org_params['67.running_mean']=mn_org_params['features.14.out.7.running_mean']
fc_org_params['67.running_var']=mn_org_params['features.14.out.7.running_var']

fc_org_params['69.beta']=mn_org_params['features.15.out.1.beta']
fc_org_params['69.gamma']=mn_org_params['features.15.out.1.gamma']
fc_org_params['69.running_mean']=mn_org_params['features.15.out.1.running_mean']
fc_org_params['69.running_var']=mn_org_params['features.15.out.1.running_var']
fc_org_params['71.weight']=mn_org_params['features.15.out.3.weight']
fc_org_params['72.beta']=mn_org_params['features.15.out.4.beta']
fc_org_params['72.gamma']=mn_org_params['features.15.out.4.gamma']
fc_org_params['72.running_mean']=mn_org_params['features.15.out.4.running_mean']
fc_org_params['72.running_var']=mn_org_params['features.15.out.4.running_var']
fc_org_params['74.weight']=mn_org_params['features.15.out.6.weight']
fc_org_params['75.beta']=mn_org_params['features.15.out.7.beta']
fc_org_params['75.gamma']=mn_org_params['features.15.out.7.gamma']
fc_org_params['75.running_mean']=mn_org_params['features.15.out.7.running_mean']
fc_org_params['75.running_var']=mn_org_params['features.15.out.7.running_var']

fc_org_params['77.beta']=mn_org_params['features.16.out.1.beta']
fc_org_params['77.gamma']=mn_org_params['features.16.out.1.gamma']
fc_org_params['77.running_mean']=mn_org_params['features.16.out.1.running_mean']
fc_org_params['77.running_var']=mn_org_params['features.16.out.1.running_var']
fc_org_params['79.weight']=mn_org_params['features.16.out.3.weight']
fc_org_params['80.beta']=mn_org_params['features.16.out.4.beta']
fc_org_params['80.gamma']=mn_org_params['features.16.out.4.gamma']
fc_org_params['80.running_mean']=mn_org_params['features.16.out.4.running_mean']
fc_org_params['80.running_var']=mn_org_params['features.16.out.4.running_var']
fc_org_params['82.weight']=mn_org_params['features.16.out.6.weight']
fc_org_params['83.beta']=mn_org_params['features.16.out.7.beta']
fc_org_params['83.gamma']=mn_org_params['features.16.out.7.gamma']
fc_org_params['83.running_mean']=mn_org_params['features.16.out.7.running_mean']
fc_org_params['83.running_var']=mn_org_params['features.16.out.7.running_var']

fc_org_params['85.beta']=mn_org_params['features.17.out.1.beta']
fc_org_params['85.gamma']=mn_org_params['features.17.out.1.gamma']
fc_org_params['85.running_mean']=mn_org_params['features.17.out.1.running_mean']
fc_org_params['85.running_var']=mn_org_params['features.17.out.1.running_var']
fc_org_params['87.weight']=mn_org_params['features.17.out.3.weight']
fc_org_params['88.beta']=mn_org_params['features.17.out.4.beta']
fc_org_params['88.gamma']=mn_org_params['features.17.out.4.gamma']
fc_org_params['88.running_mean']=mn_org_params['features.17.out.4.running_mean']
fc_org_params['88.running_var']=mn_org_params['features.17.out.4.running_var']
fc_org_params['90.weight']=mn_org_params['features.17.out.6.weight']
fc_org_params['91.beta']=mn_org_params['features.17.out.7.beta']
fc_org_params['91.gamma']=mn_org_params['features.17.out.7.gamma']
fc_org_params['91.running_mean']=mn_org_params['features.17.out.7.running_mean']
fc_org_params['91.running_var']=mn_org_params['features.17.out.7.running_var']

fc_org_params['93.beta']=mn_org_params['features.18.out.1.beta']
fc_org_params['93.gamma']=mn_org_params['features.18.out.1.gamma']
fc_org_params['93.running_mean']=mn_org_params['features.18.out.1.running_mean']
fc_org_params['93.running_var']=mn_org_params['features.18.out.1.running_var']
fc_org_params['95.weight']=mn_org_params['features.18.out.3.weight']
fc_org_params['96.beta']=mn_org_params['features.18.out.4.beta']
fc_org_params['96.gamma']=mn_org_params['features.18.out.4.gamma']
fc_org_params['96.running_mean']=mn_org_params['features.18.out.4.running_mean']
fc_org_params['96.running_var']=mn_org_params['features.18.out.4.running_var']
fc_org_params['98.weight']=mn_org_params['features.18.out.6.weight']
fc_org_params['99.beta']=mn_org_params['features.18.out.7.beta']
fc_org_params['99.gamma']=mn_org_params['features.18.out.7.gamma']
fc_org_params['99.running_mean']=mn_org_params['features.18.out.7.running_mean']
fc_org_params['99.running_var']=mn_org_params['features.18.out.7.running_var']

fc_org_params['101.beta']=mn_org_params['features.19.out.1.beta']
fc_org_params['101.gamma']=mn_org_params['features.19.out.1.gamma']
fc_org_params['101.running_mean']=mn_org_params['features.19.out.1.running_mean']
fc_org_params['101.running_var']=mn_org_params['features.19.out.1.running_var']
fc_org_params['103.weight']=mn_org_params['features.19.out.3.weight']
fc_org_params['104.beta']=mn_org_params['features.19.out.4.beta']
fc_org_params['104.gamma']=mn_org_params['features.19.out.4.gamma']
fc_org_params['104.running_mean']=mn_org_params['features.19.out.4.running_mean']
fc_org_params['104.running_var']=mn_org_params['features.19.out.4.running_var']
fc_org_params['106.weight']=mn_org_params['features.19.out.6.weight']
fc_org_params['107.beta']=mn_org_params['features.19.out.7.beta']
fc_org_params['107.gamma']=mn_org_params['features.19.out.7.gamma']
fc_org_params['107.running_mean']=mn_org_params['features.19.out.7.running_mean']
fc_org_params['107.running_var']=mn_org_params['features.19.out.7.running_var']
fc_org_params['108.weight']=mn_org_params['features.20.weight']
fc_org_params['109.beta']=mn_org_params['features.21.beta']
fc_org_params['109.gamma']=mn_org_params['features.21.gamma']
fc_org_params['109.running_mean']=mn_org_params['features.21.running_mean']
fc_org_params['109.running_var']=mn_org_params['features.21.running_var']
fc_org_params['112.weight']=mn_org_params['output.0.weight']

In [16]:
mx.nd.save("process/mobilenet_v2_0_5_s16.params",s16_org_params)

In [17]:
mx.nd.save("process/mobilenet_v2_0_5_s32.params",s32_org_params)

In [18]:
mx.nd.save("process/mobilenet_v2_0_5_fc.params",fc_org_params)